# 1. Imports

In [1]:
import pandas as pd
from sklearn import preprocessing
import numpy as np
from sklearn.feature_selection import VarianceThreshold

# 2. Importing the Datasets

In [2]:
ddos_portmap_2018_df = pd.read_csv("../data/train_test/CSE_CIC_IDS2018/ddos_portmap_2018_train.csv", index_col=0)

In [3]:
ddos_ldap_2019_df = pd.read_csv("../data/train_test/CIC_DDoS2019/ddos_ldap_2019_train.csv", index_col=0)

In [4]:
ddos_netbios_2019_df = pd.read_csv("../data/train_test/CIC_DDoS2019/ddos_netbios_2019_train.csv", index_col=0)

In [5]:
ddos_syn_2019_df = pd.read_csv("../data/train_test/CIC_DDoS2019/ddos_syn_2019_train.csv", index_col=0)

In [6]:
ddos_udp_2019_df = pd.read_csv("../data/train_test/CIC_DDoS2019/ddos_udp_2019_train.csv", index_col=0)

In [7]:
target_feature = 'Label'

# 3. Feature Selection - Variance Threshold

### Helper Functions

In [8]:
def VarianceThreshold_Helper(X_train, threshold):
    
    new_X_train = X_train.copy()

    constant_filter = VarianceThreshold(threshold= threshold)
    constant_filter.fit(new_X_train)

    len(new_X_train.columns[constant_filter.get_support()])

    constant_columns = [column for column in new_X_train.columns
                        if column not in new_X_train.columns[constant_filter.get_support()]]

    new_X_train.drop(labels=constant_columns, axis=1, inplace=True)
    
    return new_X_train.columns.tolist()

In [9]:
def stable_VarianceThreshold(train_df, num_splits, threshold, verbose):
    
    selectedFeatures = []
    
    df_split = np.array_split(train_df, num_splits)
    
    for small_df in df_split:
        X_train = small_df.drop([target_feature], axis=1)
        y_train = small_df[target_feature]
        
        selectedFeatures = selectedFeatures + VarianceThreshold_Helper(X_train, threshold)
        # NOTE TO ARYAN, PRANAV, AND ANISHA: THIS IS THE LINE YOU SHOULD CHANGE, AFTER IMPLEMENTING YOUR FEATURE
        # SELECTION METHOD
        
    rank_data = {}
    features = train_df.columns.tolist()
    ranks = []
        
    for feature in features:
        if verbose :
            print("Feature: "+feature+". Count: "+str(selectedFeatures.count(feature))+"/"+str(num_splits))
        ranks.append(selectedFeatures.count(feature)/num_splits)
    
    rank_data = {'feature':features, 'rank':ranks}
    rank_df = pd.DataFrame(rank_data) 
    
    return rank_df

### Investigating a Good Choice of Threshold

In [10]:
X_train_portmap = ddos_portmap_2018_df.drop([target_feature], axis=1)
y_train_portmap = ddos_portmap_2018_df[target_feature]

In [11]:
X_train_portmap.columns

Index(['DestinationPort', 'FlowDuration', 'TotalFwdPackets',
       'TotalBackwardPackets', 'TotalLengthofFwdPackets',
       'TotalLengthofBwdPackets', 'FwdPacketLengthMax', 'FwdPacketLengthMin',
       'FwdPacketLengthMean', 'FwdPacketLengthStd', 'BwdPacketLengthMax',
       'BwdPacketLengthMin', 'BwdPacketLengthMean', 'BwdPacketLengthStd',
       'FlowBytes/s', 'FlowPackets/s', 'FlowIATMean', 'FlowIATStd',
       'FlowIATMax', 'FlowIATMin', 'FwdIATTotal', 'FwdIATMean', 'FwdIATStd',
       'FwdIATMax', 'FwdIATMin', 'BwdIATTotal', 'BwdIATMean', 'BwdIATStd',
       'BwdIATMax', 'BwdIATMin', 'FwdPSHFlags', 'BwdPSHFlags', 'FwdURGFlags',
       'BwdURGFlags', 'FwdHeaderLength', 'BwdHeaderLength', 'FwdPackets/s',
       'BwdPackets/s', 'MinPacketLength', 'MaxPacketLength',
       'PacketLengthMean', 'PacketLengthStd', 'PacketLengthVariance',
       'FINFlagCount', 'SYNFlagCount', 'RSTFlagCount', 'PSHFlagCount',
       'ACKFlagCount', 'URGFlagCount', 'CWEFlagCount', 'ECEFlagCount',
       '

In [12]:
for t in [0.00001, 0.0001, 0.001, 0.01, 0.02, 0.05, 0.1, 0.2, 0.3]:
    features = VarianceThreshold_Helper(X_train = X_train_portmap, threshold = t)
    print("Variance Threshold "+str(t)+": "+". Features: ("+str(len(features))+") "+str(features))
    print("_______________________________________________________________________________")

Variance Threshold 1e-05: . Features: (68) ['DestinationPort', 'FlowDuration', 'TotalFwdPackets', 'TotalBackwardPackets', 'TotalLengthofFwdPackets', 'TotalLengthofBwdPackets', 'FwdPacketLengthMax', 'FwdPacketLengthMin', 'FwdPacketLengthMean', 'FwdPacketLengthStd', 'BwdPacketLengthMax', 'BwdPacketLengthMin', 'BwdPacketLengthMean', 'BwdPacketLengthStd', 'FlowBytes/s', 'FlowPackets/s', 'FlowIATMean', 'FlowIATStd', 'FlowIATMax', 'FlowIATMin', 'FwdIATTotal', 'FwdIATMean', 'FwdIATStd', 'FwdIATMax', 'FwdIATMin', 'BwdIATTotal', 'BwdIATMean', 'BwdIATStd', 'BwdIATMax', 'BwdIATMin', 'FwdPSHFlags', 'FwdHeaderLength', 'BwdHeaderLength', 'FwdPackets/s', 'BwdPackets/s', 'MinPacketLength', 'MaxPacketLength', 'PacketLengthMean', 'PacketLengthStd', 'PacketLengthVariance', 'FINFlagCount', 'SYNFlagCount', 'RSTFlagCount', 'PSHFlagCount', 'ACKFlagCount', 'URGFlagCount', 'ECEFlagCount', 'Down/UpRatio', 'AveragePacketSize', 'AvgFwdSegmentSize', 'AvgBwdSegmentSize', 'FwdHeaderLength.1', 'SubflowFwdPackets', 'S

ValueError: No feature in X meets the variance threshold 0.30000

# 4. Saving the Rankings

In [13]:
portmap_ranking = stable_VarianceThreshold(ddos_portmap_2018_df, 10, 0.00005, verbose=False)
portmap_ranking.to_csv("../ranking/CSE_CIC_IDS2018/ddos_portmap_2018_VarianceThreshold.csv")
portmap_ranking.head(20)

,feature,rank
0,DestinationPort,1.0
1,FlowDuration,1.0
2,TotalFwdPackets,0.3
3,TotalBackwardPackets,0.2
4,TotalLengthofFwdPackets,1.0
5,TotalLengthofBwdPackets,0.3
6,FwdPacketLengthMax,1.0
7,FwdPacketLengthMin,1.0
8,FwdPacketLengthMean,1.0
9,FwdPacketLengthStd,1.0


In [14]:
ldap_ranking = stable_VarianceThreshold(ddos_ldap_2019_df, 10, 0.00005, verbose=False)
ldap_ranking.to_csv("../ranking/CIC_DDoS2019/ddos_ldap_2019_VarianceThreshold.csv")
ldap_ranking.head(20)

,feature,rank
0,FlowID,1.0
1,SourceIP,1.0
2,SourcePort,1.0
3,DestinationIP,1.0
4,DestinationPort,1.0
5,Protocol,1.0
6,FlowDuration,1.0
7,TotalFwdPackets,0.5
8,TotalBackwardPackets,0.1
9,TotalLengthofFwdPackets,0.1


In [15]:
netbios_ranking = stable_VarianceThreshold(ddos_netbios_2019_df, 10, 0.00005, verbose=False)
netbios_ranking.to_csv("../ranking/CIC_DDoS2019/ddos_netbios_2019_VarianceThreshold.csv")
netbios_ranking.head(20)

,feature,rank
0,FlowID,1.0
1,SourceIP,1.0
2,SourcePort,1.0
3,DestinationIP,1.0
4,DestinationPort,1.0
5,Protocol,1.0
6,FlowDuration,1.0
7,TotalFwdPackets,0.0
8,TotalBackwardPackets,0.1
9,TotalLengthofFwdPackets,0.3


In [16]:
syn_ranking = stable_VarianceThreshold(ddos_syn_2019_df, 10, 0.00005, verbose=False)
syn_ranking.to_csv("../ranking/CIC_DDoS2019/ddos_syn_2019_VarianceThreshold.csv")
syn_ranking.head(20)

,feature,rank
0,FlowID,1.0
1,SourceIP,1.0
2,SourcePort,1.0
3,DestinationIP,1.0
4,DestinationPort,1.0
5,Protocol,1.0
6,FlowDuration,1.0
7,TotalFwdPackets,0.0
8,TotalBackwardPackets,0.1
9,TotalLengthofFwdPackets,0.3


In [17]:
udp_ranking = stable_VarianceThreshold(ddos_udp_2019_df, 10, 0.00005, verbose=False)
udp_ranking.to_csv("../ranking/CIC_DDoS2019/ddos_udp_2019_VarianceThreshold.csv")
udp_ranking.head(20)

,feature,rank
0,FlowID,1.0
1,SourceIP,1.0
2,SourcePort,1.0
3,DestinationIP,1.0
4,DestinationPort,1.0
5,Protocol,1.0
6,FlowDuration,1.0
7,TotalFwdPackets,1.0
8,TotalBackwardPackets,1.0
9,TotalLengthofFwdPackets,1.0
